In [28]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
from functionsMaths import calculateDelta, get_mse,gradiente_descendente, feedforward, div_tuplas, init_params_test
import os
import math
from statistics import mean

In [30]:
#Cantidad de letras por cada set
cantidad =['100','500','1000']

#Porcentajes datos de validacion
set_validation = [0.1,0.2,0.3]

#Cantidad de epocas
epocas = 1

#Pruebas solicitadas
#1er elemento corresponde a cantidad de neuronas por capas
    #[5] = 5 neuronas con 1 capa oculta 
    #[5,5] = 5 neuronas con 2 capas ocultas
#2do porcentaje de aprendizaje
#3ro porcentaje de momento
test =[ [[5],0.5,0.5], [[10],0.5,0.5], [[5,5],0.5,0.5], [[10,10],0.5,0.5]]

In [13]:
rows = []
for nro in range(len(test)):
    #architecture
    W,B = init_params_test(test[nro][0])
    lr = np.array(test[nro][1])
    m = np.array(test[nro][2])
    Y = []
    for set in range (len(cantidad)):
        for percentage in range(len(set_validation)):
        
            data = (pd.read_csv(os.path.join(os.path.abspath(''),"data","distorsionadas",cantidad[set],'letras.csv'),sep=';',header=None)).to_numpy()
            #Convertimos el dataframe en array numpy
            data_train = data[:int(len(data)-int(len(data)*set_validation[percentage])*2)]
            data_validation = data[int(len(data)-int(len(data)*set_validation[percentage])*2):int(len(data)-int(len(data)*set_validation[percentage])*2)+int(len(data)*set_validation[percentage])]
            data_test = data[int(len(data)-int(len(data)*set_validation[percentage])*2)+int(len(data)*set_validation[percentage]):]

            #Division de datos en tuplas(datos,clase)
            letras_train = div_tuplas(data_train)
            letras_test = div_tuplas(data_train)
            letras_validation = div_tuplas(data_validation)


            #Comienzo de Perceptron
            err_train_epoc = 0
            err_valid_epoc = 0

            for epoc in range(epocas):
                err_training = np.array([0,0,0])
                err_validation = np.array([0,0,0])

                #TRAINING
                for i in range(len(letras_train)):
        
                    Y = feedforward(letras_train[i][0],W,B,len(test[nro][0])+2)
                    Ye = np.array(letras_train[i][1])
                    err_training = err_training + ((Ye-Y[-1])**2)

                    #Deltas
                    deltas = calculateDelta(Y[-1],Ye,W[1:])

                    W,B = gradiente_descendente(W,B,deltas,Y[0:len(Y)-1],lr,m)

                #VALIDATION
                for i in range(len(letras_validation)):
                    Y = feedforward(letras_validation[i][0],W,B,len(test[nro][0])+2)
                    Ye = np.array(letras_validation[i][1])
                err_validation = err_validation + ((Ye-Y[-1])**2)

            err_train_epoc = get_mse(np.mean(err_training),len(letras_train))
            err_valid_epoc = get_mse(np.mean(err_validation),len(letras_validation))
            rows.append([len(test[nro][0]),test[nro][0][0],epoc,cantidad[set],str(set_validation[percentage]*100)+' %',err_train_epoc,err_valid_epoc])
    
     

In [38]:
tabla = pd.DataFrame(rows,columns=['count layer hidden','count neuron','epoc','count data','percentage validation','error training','error validation'])
tabla.groupby('epoc','count neuron').sum()

ValueError: No axis named count neuron for object type DataFrame